<a href="https://colab.research.google.com/github/YoonSungLee/AI-Innovation-Square_Vision/blob/master/Day%2010_Object%20detection%20%EC%9D%91%EC%9A%A9%20%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8%20%EA%B0%9C%EB%B0%9C%ED%95%98%EA%B8%B0/Cifar10_Challenge_callback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

import os

Using TensorFlow backend.


In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My Drive/Colab Notebooks/models

/content/gdrive/My Drive/Colab Notebooks/models


In [0]:
batch_size = 16
num_classes = 10
epochs = 25

In [0]:
# CIFAR-10 : class 별 비율이 비슷한 모델
# 실전에서는 class 별 비율이 중요하고 따라서 ImageDataGenerator를 이용하거나 많은 데이터가 필요하다

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes) # One-hot Encoding
y_test = keras.utils.to_categorical(y_test, num_classes) # One-hot Encoding

In [0]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # 정규화
x_test /= 255 # 정규화

In [0]:
print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

Using real-time data augmentation.


In [0]:
filepath = "./saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

네트워크를 직접 설계해 보세요!

In [0]:
# for example,
n_kernels = 32
model = Sequential()

# N, 32, 32, 3
model.add(Conv2D(n_kernels, (3,3), activation='relu', kernel_initializer='he_normal', padding='valid', input_shape=(32, 32, 3)))
# N, 30, 30, 32
model.add(Conv2D(n_kernels*2, (3, 3), activation='relu', kernel_initializer='he_normal', padding='valid'))
# N, 28, 28, 64
model.add(BatchNormalization())
# N, 28, 28, 64
model.add(MaxPooling2D())

# N, 14, 14, 64
model.add(Conv2D(n_kernels*2, (3,3), activation='relu', kernel_initializer='he_normal', padding='valid'))
# N, 12, 12, 64
model.add(Conv2D(n_kernels*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='valid'))
# N, 10, 10, 128
model.add(BatchNormalization())
# N, 10, 10, 128
model.add(MaxPooling2D())

# N, 5, 5, 128
model.add(Conv2D(n_kernels*8, (3,3), activation='relu', kernel_initializer='he_normal', padding='valid'))
# N, 3, 3, 256
model.add(BatchNormalization())

# N, 3, 3, 256
model.add(GlobalAveragePooling2D()) # 기존의 CNN에서 feature map을 flatten하는 방식이 비효율적이라고 판단하여 새롭게 등장한 방법(각 feature map의 평균을 구해서 flatten처럼 길게 배열)
# N, 256
model.add(Dense(10, activation='softmax', kernel_initializer='glorot_uniform'))
# N, 10

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 10, 10, 128)      

In [0]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [0]:
# Let's train the model using RMSprop
model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4, callbacks=[checkpoint])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
3125/3125 [==============================] - 49s 16ms/step - loss: 1.4630 - acc: 0.4798 - val_loss: 1.4850 - val_acc: 0.4973

Epoch 00001: saving model to ./saved-model-01-0.50.hdf5
Epoch 2/25
3125/3125 [==============================] - 41s 13ms/step - loss: 1.1853 - acc: 0.5843 - val_loss: 1.1230 - val_acc: 0.6105

Epoch 00002: saving model to ./saved-model-02-0.61.hdf5
Epoch 3/25
3125/3125 [==============================] - 41s 13ms/step - loss: 1.0782 - acc: 0.6266 - val_loss: 0.9726 - val_acc: 0.6567

Epoch 00003: saving model to ./saved-model-03-0.66.hdf5
Epoch 4/25
3125/3125 [==============================] - 40s 13ms/step - loss: 0.9963 - acc: 0.6533 - val_loss: 0.9691 - val_acc: 0.6731

Epoch 00004: saving model to ./saved-model-04-0.67.hdf5
Epoch 5/25
3125/3125 [==============================] - 41s 13ms/step - loss: 0.9428 - acc: 0.6750 - val_loss: 0.9209 - val_acc: 0.6